# Predicción del *default* usando redes neuronales

En este documento se desarrollará el ajuste de un modelo de **regresión con redes neuronales
** con miras a predecir el *default*, a partir de un conjunto de características.

# 1. Limpieza de los datos

Realice la limpieza de los datasets:

- Renombre la columna "default payment next month" a "default".
- Remueva la columna "ID".
- Elimine los registros con informacion no disponible.
- Para la columna EDUCATION, valores > 4 indican niveles superiores de educación, agrupe estos valores en la categoría "others".
- Renombre la columna "default payment next month" a "default"
- Remueva la columna "ID".

## 1.0. Lectura de los datos

Se definirá una función que realice la lectura de los datos.


In [1]:
#
# Paquetes necesarios
#

# Paquetes para generales y para la manipulación de datoss
import pandas as pd
pd.set_option('display.notebook_repr_html', False)
import numpy as np

# Paquetes para trabajo con carpetas
import os
import gzip
import json

# Paquetes analíticos
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, precision_score, balanced_accuracy_score, recall_score, f1_score, confusion_matrix

In [2]:
#
# Lectura de los datos
#

def lectura(direccion: str) -> pd.DataFrame:
    """
    Lee los datos

    Args:
        direccion: dirección del archivo que contiene los datos
    
    Returns:
        Marco de datos de pandas
    """

    df = pd.read_csv(
        direccion,
        header=0,
        encoding='utf-8',
        compression='zip'
    )

    return df

In [3]:
direccion_ts = '../files/input/test_data.csv.zip'
direccion_tr = '../files/input/train_data.csv.zip'

df_ts = lectura(direccion_ts)
df_tr = lectura(direccion_tr)

In [4]:
print(
    "Dimensiones <<<<<<",
    "Train:",
    df_tr.shape,
    "",
    "------",
    "Test:",
    df_ts.shape,
    sep='\n',
)

Dimensiones <<<<<<
Train:
(21000, 25)

------
Test:
(9000, 25)


Nótese que el marco de datos contiene 25 columna y 21 mil observaciones para el conjunto de entrenamiento y 9 mil para el de validación. Ahora, se mostrarán las primeras y últimas tres observaciones de cada marco de datos:

In [5]:
print(
    "Primeras observaciones en cada registro <<<<<<",
    "Train:",
    pd.concat([df_tr.head(3), df_tr.tail(3)], axis = 0),
    "",
    "------------------------------------",
    "------------------------------------",
    "------------------------------------",
    "Test:",
    pd.concat([df_ts.head(3), df_ts.tail(3)], axis = 0),
    sep='\n',
)

Primeras observaciones en cada registro <<<<<<
Train:
          ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  \
0      10748     310000    1          3         1   32      0      0      0   
1      12574      10000    2          3         1   49     -1     -1     -2   
2      29677      50000    1          2         1   28     -1     -1     -1   
20997     26      50000    1          3         2   23      0      0      0   
20998  14778      90000    2          3         2   25      0      0      0   
20999  20634     120000    1          2         2   31      0      0      0   

       PAY_4  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  \
0          0  ...      84373      57779      14163      8295      6000   
1         -1  ...       1690       1138        930         0         0   
2          0  ...      45975       1300      43987         0     46257   
20997      0  ...      28967      29829      30046      1973      1426   
20998      0  ...     

No ha signos de desfase o error de la lectura en el marco de datos. Ahora se observa el tipo de dato de cada una de las 25 columnas del marco de datos:

In [6]:
print(
    "Tipo de datos <<<<<<",
    "Train:",
    df_tr.dtypes,
    "",
    "------------------------------------",
    "------------------------------------",
    "------------------------------------",
    "Test:",
    df_ts.dtypes,
    sep='\n',
)

Tipo de datos <<<<<<
Train:
ID                            int64
LIMIT_BAL                     int64
SEX                           int64
EDUCATION                     int64
MARRIAGE                      int64
AGE                           int64
PAY_0                         int64
PAY_2                         int64
PAY_3                         int64
PAY_4                         int64
PAY_5                         int64
PAY_6                         int64
BILL_AMT1                     int64
BILL_AMT2                     int64
BILL_AMT3                     int64
BILL_AMT4                     int64
BILL_AMT5                     int64
BILL_AMT6                     int64
PAY_AMT1                      int64
PAY_AMT2                      int64
PAY_AMT3                      int64
PAY_AMT4                      int64
PAY_AMT5                      int64
PAY_AMT6                      int64
default payment next month    int64
dtype: object

------------------------------------
--------------------

Todas las columnas del marco de datos son de tipo `int64`. De acuerdo con la documentación de la tabla, algunas columnas son categóricas, por lo que más adelante se abordará la corrección.

## 1.1. Renombrar la columna `default`

In [7]:
def renombre_columna(
        df: pd.DataFrame, nombre_inic: str, nombre_fin: str
) -> None:
    """
    Renombra el nombre de una columna
    
    Args:
        df: marco de datos
        nombre_inic: nombre que tiene la columna actualmente
        nombre_fin: nombre que se quiere tenga la columna

    Returns:
        No retorna nada. Los cambios se aplican sobre el marco
        de datos entregado
    """

    df.rename(
        columns={
            nombre_inic: nombre_fin,
        },
        inplace=True,
    )


In [8]:
renombre_columna(df_tr, 'default payment next month', 'default')
renombre_columna(df_ts, 'default payment next month', 'default')

# Chequeo

print(
    "Nombre de las columnas <<<<<<",
    "Train:",
    df_tr.columns,
    "",
    "------------------------------------",
    "------------------------------------",
    "------------------------------------",
    "Test:",
    df_ts.columns,
    sep='\n',
)

Nombre de las columnas <<<<<<
Train:
Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')

------------------------------------
------------------------------------
------------------------------------
Test:
Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')


## 1.2. Remoción de la columna `ID`

In [9]:
def remover_columna(
        df: pd.DataFrame, columna: str
) -> None:
    """
    Elimina una columna determinada

    Args:
        df: Marco de datos en el que se eliminará
        una columna
        columna: Columna a eliminar del marco de datos
    
    Raises:
        KeyError: La columna no existe en el marco de datos
    """
    try:
        df.drop(
            columns=columna,
            axis=1,
            inplace=True,
        )
    except KeyError:
        print(f"La columna '{columna}' no existe en el marco de datos.")

In [10]:
remover_columna(df_tr, 'ID')
remover_columna(df_ts, 'ID')

# Chequeo

print(
    "Nombre de las columnas <<<<<<",
    "Train:",
    df_tr.columns,
    "",
    "------------------------------------",
    "------------------------------------",
    "------------------------------------",
    "Test:",
    df_ts.columns,
    sep='\n',
)

Nombre de las columnas <<<<<<
Train:
Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')

------------------------------------
------------------------------------
------------------------------------
Test:
Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')


## 1.3. Eliminar los registros con información no disponible

Revisemos la cantidad de registros nulos para cada caso:

In [11]:
print(
    "Registros nulos <<<<<<",
    "Train:",
    df_tr.isna().sum(),
    "",
    "------------------------------------",
    "------------------------------------",
    "------------------------------------",
    "Test:",
    df_tr.isna().sum(),
    sep='\n',
)

Registros nulos <<<<<<
Train:
LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
default      0
dtype: int64

------------------------------------
------------------------------------
------------------------------------
Test:
LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
default      0
dtype: int64


De acuerdo con lo anterior, no se cuentan con registros nulos en el marco de datos; sin embargo, al leer la documentación de las tablas, se obtiene que para las columnas `MARRIAGE` y `EDUCATION` el nivel `'0'` hace referencia a registros nulos. Así, se eliminarán aquellos registros que sean iguales a cero para este par de columnas.

In [12]:
def eliminacion_especial(
        df: pd.DataFrame, columna_categorica: str, valor: int
) -> pd.DataFrame:
    """
    Elimina los registros de un marco de datos que sean iguales
    a un valor determinado en una columna.

    Args:
        df: Marco de datos
        columna_categorica: columna sobre la que se realiza la
        inspección
        valor: valor que quiere descartarse

    Returns:
        Marco de datos de pandas
    """

    return df.loc[df[columna_categorica] != valor]

In [13]:
df_ts['MARRIAGE'].value_counts()

MARRIAGE
2    4732
1    4154
3      98
0      16
Name: count, dtype: int64

In [14]:
# Para el conjunto de entrenamiento
df_tr = eliminacion_especial(df_tr, 'MARRIAGE', 0)
df_tr = eliminacion_especial(df_tr, 'EDUCATION', 0)

# Para el conjunto de entrenamiento
df_ts = eliminacion_especial(df_ts, 'MARRIAGE', 0)
df_ts = eliminacion_especial(df_ts, 'EDUCATION', 0)

Para revisar que el proceso se haya dado de forma adecuada, se revisarán las dimensiones de los dos marcos de datos:

In [15]:
print(
    "Dimensiones <<<<<<",
    "Train:",
    df_tr.shape,
    "",
    "------",
    "Test:",
    df_ts.shape,
    sep='\n',
)

Dimensiones <<<<<<
Train:
(20953, 24)

------
Test:
(8979, 24)


Nótese que es inferior a la cantidad de registros que se tenía inicialmente para ambos marcos de datos. También se revisará el conteo de registros para este par de columnas:

In [16]:
print(
    "Tabla de frecuencias <<<<<<",
    "Train: ========================",
    "Variable MARRIAGE:",
    df_tr.MARRIAGE.value_counts(),
    "Variable EDUCATION:",
    df_tr.EDUCATION.value_counts(),
    "------------------------------------",
    "------------------------------------",
    "------------------------------------",
    "Test: ========================",
    "Variable MARRIAGE:",
    df_ts.MARRIAGE.value_counts(),
    "Variable EDUCATION:",
    df_ts.EDUCATION.value_counts(),
    sep='\n',
)

Tabla de frecuencias <<<<<<
Train: ========================
Variable MARRIAGE:
MARRIAGE
2    11226
1     9502
3      225
Name: count, dtype: int64
Variable EDUCATION:
EDUCATION
2    9756
1    7476
3    3396
5     187
4      98
6      40
Name: count, dtype: int64
------------------------------------
------------------------------------
------------------------------------
Test: ========================
Variable MARRIAGE:
MARRIAGE
2    4728
1    4153
3      98
Name: count, dtype: int64
Variable EDUCATION:
EDUCATION
2    4268
1    3105
3    1477
5      93
4      25
6      11
Name: count, dtype: int64


En efecto, para ningún caso, se cuenta con ningún registro que tenga el nivel `0` para este par de variables.

## 1.4. Refactorización de la columna `EDUCATION`

Se quiere que los niveles superiores a `4` se refactoricen o recategoricen como '`others`'. De acuerdo con la documentación, este nivel está representado por `4`, por lo que basta con mandar a cualquier valor mayor a `4` a este valor. Para esto se puede usar una función lambda.

In [17]:
def recategorizacion(
        df: pd.DataFrame, columna: str, umbral: int, referencia: int
) -> None:
    """
    Aquellos valores mayores a cierto umbral para una columna
    serán recategorizados de acuerdo a un valor de referencia

    Args:
        df: Marco de datos de pandas
        columna: Columna sobre la cual se quiere realizar la aplicación
        umbral: Valor por encima del cual se quiere realizar la recategorización
        referencia: Valor al cual se deben recategorizar los valores que superen
        el umbral.
    Returns:
        Se realiza la aplicación directamente sobre el marco de datos
    """

    df[columna] = df[columna].apply(
        lambda x: referencia if x > umbral else x
    )

In [18]:
recategorizacion(
    df=df_tr, columna='EDUCATION', umbral=4, referencia=4
)

recategorizacion(
    df=df_ts, columna='EDUCATION', umbral=4, referencia=4
)

#
# Chequeo
#

print(
    "Refactorización de EDUCATION <<<<<<",
    "Train:",
    df_tr['EDUCATION'].value_counts(),
    "",
    "------------------------------------",
    "------------------------------------",
    "------------------------------------",
    "Test:",
    df_ts['EDUCATION'].value_counts(),
    sep='\n',
)

Refactorización de EDUCATION <<<<<<
Train:
EDUCATION
2    9756
1    7476
3    3396
4     325
Name: count, dtype: int64

------------------------------------
------------------------------------
------------------------------------
Test:
EDUCATION
2    4268
1    3105
3    1477
4     129
Name: count, dtype: int64


# 2. División en conjuntos de entrenamiento y validación

Como ya se entregaron los datos separados en un par de conjuntos de entrenamiento y validación, basta con realizar diferenciar para cada caso la variable objetivo de la matriz de diseño.

In [19]:
# Para el conjunto de entrenamiento
y_tr = df_tr.pop('default')
X_tr = df_tr.copy()

# Para el conjunto de validación
y_ts = df_ts.pop('default')
X_ts = df_ts.copy()

# 3. *Pipeline*

Se quiere construir un *pipeline* que, para el ajuste del modelo, realice los siguientes pasos:
- Binarice las variables categóricas usando *One-Hot-Encoder*
- Escale las variables cuantitativas al intervalo $\left[0, \ 1 \right]$.
- Seleccione las $K$ mejores características
- Ajuste un modelo de regresión logística.

In [20]:
#Columnas categoricas
categorical_features=["SEX","EDUCATION","MARRIAGE"]
numerical_features=num_columns = [col for col in X_tr.columns if col not in categorical_features]
#print(numerical_features)

#preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
        ('scaler',StandardScaler(),numerical_features),
    ],
    #remainder=StandardScaler(),
)

#pipeline
pipeline=Pipeline(
    [
        ("preprocessor",preprocessor),
        ('feature_selection',SelectKBest(score_func=f_classif)),
        ('pca',PCA()),
        ('classifier',MLPClassifier(max_iter=15000, early_stopping=True))
    ]
)

# 4. Optimización de hiperparámentros

Se quiere realizar el ajuste del modelo optimizando *hiperparámentros*, para lo cual se va a usar **validación cruzada** con 10 divisiones del marco de entrenamiento. Para determinar qué hiperparámetros son los mejores, se va a emplear como métrica la *precisión balanceada*.

In [21]:
# Malla de hiperparámetros
param_grid = {
    'pca__n_components': [20],
    'feature_selection__k': [20],
    'classifier__hidden_layer_sizes': [(50,30,40,60)],
    'classifier__alpha': [0.256],
    'classifier__learning_rate': ['adaptive'],
    'classifier__activation': ['relu'],
    'classifier__solver': ['adam'],
    'classifier__learning_rate_init': [0.001]
}

modelo = GridSearchCV(
    pipeline,
    param_grid,
    cv=10,
    scoring='balanced_accuracy',
    n_jobs=10,
)

modelo

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE']),
                                                                        ('scaler',
                                                                         StandardScaler(),
                                                                         ['LIMIT_BAL',
                                                                          'AGE',
                                                                          'PAY_0',
                                                                          'PAY_2',
                                                                          'PAY_3',
                                                                          'PAY_4',
                                                                          'PAY_5',
                                                                          'PAY_6',
                                                                          'BILL_AMT1',
                                                                          'BILL_AMT2',
                                                                          'BILL_AMT3',
                                                                          'BILL_AMT4',
                                                                          'BILL_AMT5',
                                                                          'BILL_AMT6',
                                                                          'PAY_AMT1',
                                                                          'PAY_AMT2',
                                                                          'PAY_AMT3',
                                                                          'PA...
                                        MLPClassifier(early_stopping=True,
                                                      max_iter=15000))]),
             n_jobs=10,
             param_grid={'classifier__activation': ['relu'],
                         'classifier__alpha': [0.256],
                         'classifier__hidden_layer_sizes': [(50, 30, 40, 60)],
                         'classifier__learning_rate': ['adaptive'],
                         'classifier__learning_rate_init': [0.001],
                         'classifier__solver': ['adam'],
                         'feature_selection__k': [20],
                         'pca__n_components': [20]},
             scoring='balanced_accuracy')

In [22]:
modelo.fit(X_tr, y_tr)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE']),
                                                                        ('scaler',
                                                                         StandardScaler(),
                                                                         ['LIMIT_BAL',
                                                                          'AGE',
                                                                          'PAY_0',
                                                                          'PAY_2',
                                                                          'PAY_3',
                                                                          'PAY_4',
                                                                          'PAY_5',
                                                                          'PAY_6',
                                                                          'BILL_AMT1',
                                                                          'BILL_AMT2',
                                                                          'BILL_AMT3',
                                                                          'BILL_AMT4',
                                                                          'BILL_AMT5',
                                                                          'BILL_AMT6',
                                                                          'PAY_AMT1',
                                                                          'PAY_AMT2',
                                                                          'PAY_AMT3',
                                                                          'PA...
                                        MLPClassifier(early_stopping=True,
                                                      max_iter=15000))]),
             n_jobs=10,
             param_grid={'classifier__activation': ['relu'],
                         'classifier__alpha': [0.256],
                         'classifier__hidden_layer_sizes': [(50, 30, 40, 60)],
                         'classifier__learning_rate': ['adaptive'],
                         'classifier__learning_rate_init': [0.001],
                         'classifier__solver': ['adam'],
                         'feature_selection__k': [20],
                         'pca__n_components': [20]},
             scoring='balanced_accuracy')

In [23]:
modelo.best_params_

{'classifier__activation': 'relu',
 'classifier__alpha': 0.256,
 'classifier__hidden_layer_sizes': (50, 30, 40, 60),
 'classifier__learning_rate': 'adaptive',
 'classifier__learning_rate_init': 0.001,
 'classifier__solver': 'adam',
 'feature_selection__k': 20,
 'pca__n_components': 20}

In [24]:
print(
    "Métricas básicas <<<<<<",
    "Train:",
    modelo.score(X_tr, y_tr),
    "",
    "------",
    "Test:",
    modelo.score(X_ts, y_ts),
    sep='\n',
)

Métricas básicas <<<<<<
Train:
0.6711809362092822

------
Test:
0.6804841327193595


# 5. Guardar el *pickle* del modelo

In [25]:
import pickle
import os
import gzip

# Carpeta donde se guarda el modelo
direccion = '../files/models'
os.makedirs(direccion, exist_ok=True)

# Guardado del modelo
direc_modelo = '../files/models/model.pkl.gz'
with gzip.open(direc_modelo, 'wb') as file:
    pickle.dump(modelo, file)

# 6. Cálculo de métricas

In [26]:
def calculo_metricas(
        modelo, y_tr, y_ts, X_tr, X_ts, directorio, nombre
):
    # Predicción
    y_pred_tr = modelo.predict(X_tr)
    y_pred_ts = modelo.predict(X_ts)

    # Métricas para entrenamiento
    metrics_train = {
        'type': 'metrics',
        'dataset': 'train',
        'precision': precision_score(y_tr, y_pred_tr),
        'balanced_accuracy': balanced_accuracy_score(y_tr, y_pred_tr),
        'recall': recall_score(y_tr, y_pred_tr),
        'f1_score': f1_score(y_tr, y_pred_tr),
    }

    # Métricas para validcion
    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',
        'precision': precision_score(y_ts, y_pred_ts),
        'balanced_accuracy': balanced_accuracy_score(y_ts, y_pred_ts),
        'recall': recall_score(y_ts, y_pred_ts),
        'f1_score': f1_score(y_ts, y_pred_ts),
    }

    # Almacenamiento
    os.makedirs(directorio, exist_ok=True)

    # Guardado de métricas
    with open(os.path.join(directorio, nombre), 'w') as file:
        file.write(json.dumps(metrics_train) + '\n')
        file.write(json.dumps(metrics_test) + '\n')

In [27]:
calculo_metricas(
    modelo=modelo,
    y_tr=y_tr,
    y_ts=y_ts,
    X_tr=X_tr,
    X_ts=X_ts,
    directorio='../files/output',
    nombre='metrics.json',
)

In [28]:
# Predicción
y_pred_tr = modelo.predict(X_tr)
y_pred_ts = modelo.predict(X_ts)

# Métricas para entrenamiento
metrics_train = {
    'type': 'metrics',
    'dataset': 'train',
    'precision': precision_score(y_tr, y_pred_tr),
    'balanced_accuracy': balanced_accuracy_score(y_tr, y_pred_tr),
    'recall': recall_score(y_tr, y_pred_tr),
    'f1_score': f1_score(y_tr, y_pred_tr),
}

# Métricas para validcion
metrics_test = {
    'type': 'metrics',
    'dataset': 'test',
    'precision': precision_score(y_ts, y_pred_ts),
    'balanced_accuracy': balanced_accuracy_score(y_ts, y_pred_ts),
    'recall': recall_score(y_ts, y_pred_ts),
    'f1_score': f1_score(y_ts, y_pred_ts),
}


In [29]:
metrics_train

{'type': 'metrics',
 'dataset': 'train',
 'precision': 0.6600763093999307,
 'balanced_accuracy': np.float64(0.6711809362092822),
 'recall': 0.40275132275132275,
 'f1_score': 0.5002628811777077}

In [30]:
metrics_test

{'type': 'metrics',
 'dataset': 'test',
 'precision': 0.6430842607313195,
 'balanced_accuracy': np.float64(0.6804841327193595),
 'recall': 0.42444910807974817,
 'f1_score': 0.511378002528445}

# 7. Matrices de confusión

In [31]:
def format_confusion_matrix(matriz, conjunto):
    return {
        'type': 'cm_matrix',
        'dataset': conjunto,
        'true_0': {
            'predicted_0': int(matriz[0, 0]),
            'predicted_1': int(matriz[0, 1])
        },
        'true_1': {
            'predicted_0': int(matriz[1, 0]),
            'predicted_1': int(matriz[1, 1])
        },
    }

In [32]:
def matriz_confusion(
        modelo, y_tr, y_ts, X_tr, X_ts, directorio, nombre
):
    
    y_tr_pred = modelo.predict(X_tr)
    y_ts_pred = modelo.predict(X_ts)

    confusion_tr = confusion_matrix(y_tr, y_tr_pred)
    confusion_ts = confusion_matrix(y_ts, y_ts_pred)

    metrics = [
        format_confusion_matrix(confusion_tr, 'train'),
        format_confusion_matrix(confusion_ts, 'test'),
    ]

    # Guardar las matrices de confusión en el mismo archivo JSON
    with open(os.path.join(directorio, nombre), 'a') as f:  # Usar 'a' para agregar después de las métricas
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')

In [33]:
matriz_confusion(
    modelo=modelo,
    y_tr=y_tr,
    y_ts=y_ts,
    X_tr=X_tr,
    X_ts=X_ts,
    directorio='../files/output',
    nombre='metrics.json',
)

In [34]:
y_tr_pred = modelo.predict(X_tr)
y_ts_pred = modelo.predict(X_ts)

confusion_tr = confusion_matrix(y_tr, y_tr_pred)
confusion_ts = confusion_matrix(y_ts, y_ts_pred)

metrics = [
    format_confusion_matrix(confusion_tr, 'train'),
    format_confusion_matrix(confusion_ts, 'test'),
]

In [35]:
confusion_tr

array([[15248,   980],
       [ 2822,  1903]])

In [36]:
confusion_ts

array([[6624,  449],
       [1097,  809]])